In [2]:
import numpy as np
import tensorflow as tf
import tflearn
import cv2

In [3]:
from kaffe.tensorflow import Network

class VGG_ILSVRC_16_layers(Network):
    def setup(self):
        (self.feed('input')
             .conv(3, 3, 64, 1, 1, name='conv1_1')
             .conv(3, 3, 64, 1, 1, name='conv1_2')
             .max_pool(2, 2, 2, 2, name='pool1')
             .conv(3, 3, 128, 1, 1, name='conv2_1')
             .conv(3, 3, 128, 1, 1, name='conv2_2')
             .max_pool(2, 2, 2, 2, name='pool2')
             .conv(3, 3, 256, 1, 1, name='conv3_1')
             .conv(3, 3, 256, 1, 1, name='conv3_2')
             .conv(3, 3, 256, 1, 1, name='conv3_3')
             .max_pool(2, 2, 2, 2, name='pool3')
             .conv(3, 3, 512, 1, 1, name='conv4_1')
             .conv(3, 3, 512, 1, 1, name='conv4_2')
             .conv(3, 3, 512, 1, 1, name='conv4_3')
             .max_pool(2, 2, 2, 2, name='pool4')
             .conv(3, 3, 512, 1, 1, name='conv5_1')
             .conv(3, 3, 512, 1, 1, name='conv5_2')
             .conv(3, 3, 512, 1, 1, name='conv5_3')
             .max_pool(2, 2, 2, 2, name='pool5')
             .fc(4096, name='fc6')
             .fc(4096, name='fc7')
             .fc(1000, relu=False, name='fc8')
             .softmax(name='prob'))

In [4]:
images = tf.placeholder(tf.float32, [None, 224, 224, 3])

In [5]:
# initialize VGG-16 net
net = VGG_ILSVRC_16_layers({'input': images})

In [28]:
# we only need to extract fc7 layer features
fc7 = net.layers['fc7']

In [29]:
def load_images(image_list):
    """Load images into numpy arrays from an image_list."""
    
    image_full_batch = []
    for image_path in image_list:
        # 1 means read color image
        image_tensor = cv2.imread(image_path, 1)
        image_full_batch.append(image_tensor)
    # put all image tensors together into the full batch
    image_full_batch = np.stack(image_tensor)
    return image_full_batch

In [19]:
image_path = "Corel5k/all_images/1000.jpg"
image_path_2 = "Corel5k/all_images/10000.jpg"
image_tensor = cv2.imread(image_path, 1)
image_t2 = cv2.imread(image_path_2, 1)
image_batch = np.stack([image_tensor, image_t2])

In [20]:
with tf.Session() as sess:
    # Load the data
    sess.run(tf.initialize_all_variables())
    net.load('vgg_16.tfmodel', sess)
    for i in xrange(1):
        train_images_batch = image_batch
        feed = {images: train_images_batch}

        image_embeddings = sess.run([fc7], feed_dict=feed)


In [27]:
image_embeddings

[array([[ 0.        ,  0.35319358,  1.00002849, ...,  1.18863904,
          0.        ,  0.30297029],
        [ 0.        ,  0.94959658,  1.84656298, ...,  0.        ,
          0.        ,  0.        ]], dtype=float32)]